In [1]:
# import libraries
import geopandas as gpd
import pydeck as pdk
import os
from dotenv import load_dotenv
load_dotenv()

/opt/homebrew/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.12.1-CAPI-1.18.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


True

In [2]:
# load parcel data from project folder (Dropbox)
df_parcels = gpd.read_file(os.environ.get('LOCAL_PATH')+"Spatial Data/ny-parcels/NYC_2021_Tax_Parcels_SHP_2203/NewYork_2021_Tax_Parcels_SHP_2203.shp")

In [3]:
# show length of dataframe
df_parcels.length

0        1921.544489
1        1200.788083
2        3741.111736
3         369.612212
4         502.254754
            ...     
42292     207.048801
42293     159.618447
42294     656.379825
42295     213.730693
42296     279.662391
Length: 42297, dtype: float64

In [4]:
# subset data frame; set to count from your image generation script
df_parcels = df_parcels.iloc[:25000] 

In [ ]:
# verify subset length
df_parcels.length

In [ ]:
# set CRS to WGS 84
df_parcels = df_parcels.to_crs(epsg=4326)

In [ ]:
# extract lat and lon from the centerpoint
df_parcels['center_point'] = df_parcels['geometry'].centroid
df_parcels['lon'] = df_parcels['center_point'].x
df_parcels['lat'] = df_parcels['center_point'].y

In [ ]:
# view df (optional)
print(df_parcels['geometry'].head())
print(df_parcels['center_point'].head())
print(df_parcels['center_point'].head())
print(df_parcels['lon'].head())
print(df_parcels['lat'].head())

In [ ]:
# display parcels as points
points = pdk.Layer(
    'ScatterplotLayer',
    data=df_parcels,
    get_position='[lon, lat]',
    get_radius=10,
    get_fill_color=[255, 0, 0],
    auto_highlight=True,
    pickable=True
)

# heatmap
heat = pdk.Layer(
    "HeatmapLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    #aggregation=pdk.types.String("MEAN"),
    aggregation=pdk.types.String("SUM"),
    get_weight=1, # Assuming equal weight for all points
    #threshold=1,
    opacity=0.9,
)

# hexagon
hex = pdk.Layer(
    "HexagonLayer",
    data=df_parcels,
    get_position=["lon", "lat"],
    radius=50,
    opacity=0.9,
    pickable=True
)

# Create the map
view = pdk.data_utils.compute_view(df_parcels[["lon", "lat"]])
view.zoom=11
view.max_zoom=20
view.min_zoom=10

r = pdk.Deck(
    layers=[hex], # choose between points, heat, or hex
    initial_view_state=view,
    #map_provider="mapbox",
    #map_style=pdk.map_styles.SATELLITE
)

# Display the visualization in the notebook (warning: increases notebook size significantly, clear output to reduce size)
# r.show()

# save the visualization to an HTML file (git-ignored)
r.to_html('parcels-map.html')